In [ ]:
!pip install streamlit  bitsandbytes accelerate pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.2 MB/s eta 0:00:00


In [ ]:
!mkdir -p /content/.streamlit


In [ ]:
!echo 'HF_TOKEN = "YOUR_HF_TOKEN_HERE"' > /content/.streamlit/secrets.toml

In [ ]:
!cat /content/.streamlit/secrets.toml


In [ ]:
%%writefile app.py

import os
from google.colab import userdata
import streamlit as st

os.environ["HF_TOKEN"] = st.secrets["HF_TOKEN"]
HUGGINGFACE_TOKEN = os.environ["HF_TOKEN"]

import requests
from PIL import Image
from huggingface_hub import InferenceClient
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time


if not HUGGINGFACE_TOKEN:
    # Eğer token hala bulunamıyorsa, kullanıcıya uyarı göster ve uygulamayı durdur.
    # Bu, LLM veya Görüntü üretme kodunun çalışmasını engeller.
    st.error(" Hata: Hugging Face API Token (HF_TOKEN) bulunamadı.")

    st.stop() # Token yoksa uygulamayı burada durdur.


@st.cache_resource
def load_mistral_model():
    """
    Loads the Mistral 7B Instruct model using 8-bit quantization.
    """
    model_id = "mistralai/Mistral-7B-Instruct-v0.2"

    # Colab GPU'su ile çalıştığından emin olmak için
    # Colab menüsünden Çalışma Zamanı -> Çalışma Zamanı Türünü Değiştir -> T4 GPU seçili olmalı!

    with st.spinner(f"Yapay Zeka Modeli Yükleniyor... ({model_id} - 8-bit Nicemleme)"):
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            load_in_8bit=True, # Sizin çalışan versiyonunuzda 8-bit yükleme vardı.
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model.eval()

    st.success("Model başarıyla yüklendi!")
    return model, tokenizer

# --- 2. Generation Function ---
def generate_response(model, tokenizer, prompt, max_len=1000):
    start_time = time.time()

    # Giriş token'larını modelin bulunduğu cihaza (GPU) taşı
    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **tokens,
            max_new_tokens=max_len,
            do_sample=True,
            temperature=0.7,
            num_beams=3,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(generated_ids[0].tolist(), skip_special_tokens=True)

    if full_output.startswith(prompt):
        response = full_output[len(prompt):].strip()
    else:
        response = full_output.strip()

    end_time = time.time()
    st.info(f"Yanıt {end_time - start_time:.2f} saniyede üretildi.")

    return response

# --- 3. Streamlit UI (Main App) ---
markdown = st.markdown("### SELECT YOUR AI MODE")
selected_mode = st.selectbox("", ["text to text", "text to image"])

if selected_mode == "text to text":
    # ------------------ TEXT-TO-TEXT LOGIC ------------------
    model, tokenizer = load_mistral_model()
    st.title("🤖 Mistral 7B Prompt Yanıtlama Uygulaması")
    st.markdown("8-bit nicemleme (quantization) ile T4 GPU'da çalışan büyük dil modeli.")

    user_prompt = st.text_area(
        "Lütfen metin isteminizi (prompt) girin:",
        value="Büyük dil modellerinin geleceği hakkında bir özet yaz.",
        height=200
    )

    if st.button("Yanıt Üret", type="primary"):
        if user_prompt:
            with st.status("Yanıt Üretiliyor...", expanded=True) as status:
                st.write("Model çıkarım yapıyor...")
                try:
                    ai_response = generate_response(model, tokenizer, user_prompt)
                    status.update(label="Yanıt Üretimi Tamamlandı!", state="complete", expanded=False)
                    st.subheader("Mistral 7B Yanıtı:")
                    st.markdown(ai_response)
                except Exception as e:
                    st.error(f"Bir hata oluştu: {e}")
                    status.update(label="Hata!", state="error")
        else:
            st.warning("Lütfen geçerli bir prompt girin.")

else: # selected_mode == "text to image"
    # ------------------ TEXT-TO-IMAGE LOGIC ------------------
    st.title("🖼️ Görüntü Üretme Uygulaması (Hugging Face API)")
    st.markdown("Stable Diffusion XL (SDXL) modelini kullanarak metinden görüntü üretin.")

    # Token'ın geçerliliğini zaten yukarıda kontrol ettik. Burada istemciyi başlatabiliriz.
    MODEL_ID = "black-forest-labs/FLUX.1-dev"
    try:
        client = InferenceClient(token=HUGGINGFACE_TOKEN)
    except Exception as e:
        st.error(f"InferenceClient başlatılamadı: {e}")
        st.stop()


    # 3. Get the prompt from user
    prompt = st.text_area(
        "Lütfen üretmek istediğiniz görüntünün İngilizce tanımını (prompt) girin:",
        value="A hyper-realistic cyborg dog wearing a space helmet, digital art, high contrast",
        height=200
    )

    negative_prompt = st.text_input(
        "Negatif Prompt (İstenmeyen Özellikler):",
        value="blurry, ugly, poorly drawn, extra limbs, watermark",
    )

    # 4. Handle Button Click
    if st.button("Görüntü Üret", type="primary"):
        if not prompt:
            st.warning("Lütfen geçerli bir prompt girin.")
            st.stop()

        with st.status("Görüntü Üretiliyor...", expanded=True) as status:
            st.write(f"Prompt: {prompt}")

            try:
                # 4. Call the free Inference API
                status.write("Hugging Face API'sine istek gönderiliyor. Bu işlem 30-60 saniye sürebilir...")

                image = client.text_to_image(
                    model=MODEL_ID,
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    guidance_scale=9,
                    width=1024,
                    height=1024
                )

                status.write("Görüntü indiriliyor...")

                # 5. Display the image
                st.subheader("Üretilen Görüntü:")
                # Doğrudan PIL Image objesini st.image'a iletiyoruz.
                st.image(
                    image,
                    caption=prompt,
                    use_column_width=True
                )

                status.update(label="Görüntü Başarıyla Üretildi!", state="complete", expanded=False)

            except requests.HTTPError as e:
                if "rate limit" in str(e):
                    st.error("**Hata:** Ücretsiz hız limitine ulaştınız. Lütfen bir süre sonra tekrar deneyin.")
                elif "Repository Not Found" in str(e):
                    st.error(" API Hatası: Model ID'si hatalı veya lisansı kabul edilmemiş. Hugging Face'de lisansı kabul edin.")
                else:
                    st.error(f" Beklenmedik API Hatası: {e}")
                status.update(label="Hata!", state="error")
            except Exception as e:
                st.error(f"Beklenmedik bir hata oluştu: {e}")
                status.update(label="Hata!", state="error")

Writing app.py


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN_HERE")
# Streamlit uygulamasını arka planda başlat
!streamlit run app.py &>/dev/null&

# Streamlit'in çalıştığı 8501 portuna tünel oluştur
public_url = ngrok.connect('8501')

# URL'yi yazdır
print(f"Uygulamanızın Genel URL'si: {public_url}")

Uygulamanızın Genel URL'si: NgrokTunnel: "https://semiannual-vida-livelily.ngrok-free.dev" -> "http://localhost:8501"
